In [46]:
import openai
import pandas as pd
import json
import os
import json
import heapq
from dataclasses import dataclass, field
from typing import List, Tuple, Dict, Optional
import sys

In [68]:
#testing the chatGPT model vs. A* search algorithm for a number of randomly generated mazes

path = r''

count = 0
total = len(os.listdir(path))
for maze in os.listdir(path):
    full_path = os.path.join(path, maze)
    print(full_path)
    test1 = run_A_STAR(maze_file, a_star_output)
    test2 = run_chatGPT(maze_file, gpt_output)

    normalized_output1 = test1.replace("'", '"')
    normalized_output2 = test2.replace("'", '"')

    if normalized_output1 == normalized_output2:
        count += 1 
    print()

print(f"Solved {count} out of {total}")

test_mazes\maze_0.csv
Maze loaded:
s,x,x,x,x
o,o,o,o,x
x,x,x,o,o
x,x,x,x,o
x,x,x,x,e


A*  Solution: {'instructions': ['forward', 'left', 'forward', 'forward', 'forward', 'right', 'forward', 'left', 'forward', 'right', 'forward', 'forward']}
Solution written to C:/Users/drk3/Documents/LabVIEW Data/a_star_output.json

LLM Solution: {"instructions": ["forward", "left", "forward", "forward", "forward", "right", "forward", "left", "forward", "right", "forward", "forward"]}
Solution written to C:/Users/drk3/Documents/LabVIEW Data/gpt_output.json

test_mazes\maze_1.csv
Maze loaded:
s,x,x,x,x
o,o,o,o,x
x,x,x,o,o
x,x,x,x,o
x,x,x,x,e


A*  Solution: {'instructions': ['forward', 'left', 'forward', 'forward', 'forward', 'right', 'forward', 'left', 'forward', 'right', 'forward', 'forward']}
Solution written to C:/Users/drk3/Documents/LabVIEW Data/a_star_output.json

LLM Solution: {"instructions": ["forward", "left", "forward", "forward", "forward", "right", "forward", "left", "forward", "right", "

In [65]:
maze_file = 'maze5.csv'
# define file output locations
a_star_output = r''
gpt_output = r''

test1 = run_A_STAR(maze_file, a_star_output)
test2 = run_chatGPT(maze_file, gpt_output)

normalized_output1 = test1.replace("'", '"')
normalized_output2 = test2.replace("'", '"')

print()
print("testing results ")
print(normalized_output1)
print(normalized_output2)
print(normalized_output1 == normalized_output2)



Maze loaded:
s,x,x,x,x
o,o,o,o,x
x,x,x,o,o
x,x,x,x,o
x,x,x,x,e


A*  Solution: {'instructions': ['forward', 'left', 'forward', 'forward', 'forward', 'right', 'forward', 'left', 'forward', 'right', 'forward', 'forward']}
Solution written to C:/Users/drk3/Documents/LabVIEW Data/a_star_output.json

LLM Solution: {"instructions": ["forward", "left", "forward", "forward", "forward", "right", "forward", "left", "forward", "right", "forward", "forward"]}
Solution written to C:/Users/drk3/Documents/LabVIEW Data/gpt_output.json

testing results 
{"instructions": ["forward", "left", "forward", "forward", "forward", "right", "forward", "left", "forward", "right", "forward", "forward"]}
{"instructions": ["forward", "left", "forward", "forward", "forward", "right", "forward", "left", "forward", "right", "forward", "forward"]}
True


### Fined Tuned Model
##### Model: OpenAI ChatGPT 4o-mini
##### Dataset size: ~ 9MB, 5000 5x5 maze examples

In [55]:
#define your OpenAI API key to your environment variables
openai.api_key = OPENAI_API_KEY

def run_chatGPT(maze_file, output_file):
    # Read the maze file
    try:
        maze = pd.read_csv(maze_file, header=None)
    except Exception as e:
        print(f"Error reading the maze file: {e}")
        return
    
    # print("Solving with chatGPT LLM")
    # Convert maze to string for prompt
    maze_str = maze.to_csv(index=False, header=False)
    num_rows, num_columns = maze.shape

    prompt = f"""
    Solve the given maze following the rules and building blocks provided.

    Building Blocks:
    - "forward": Moves block one cell in the direction the block is currently facing.
    - "backward": Moves block one cell in the opposite direction the block is currently facing.
    - "left": Rotates block 90 degrees counterclockwise in place (does not move cells).
    - "right": Rotates block 90 degrees clockwise in place (does not move cells).

    Maze File:
    This is a {num_rows} x {num_columns} grid where:
    - 's' represents the start cell.
    - 'e' represents the end cell.
    - 'x' represents an occupied cell.
    - 'o' represents an open path.

    Here is the maze you need to solve:
    {maze_str}

    Rules:
    1. The block starts at the 's' cell facing down (The start cell is (0,0) and is facing in the direction of (1,0)).
    2. You can only move into 'o' cells. You cannot move into 'x' cells.
    3. The block cannot move outside the maze boundaries.
    4. The goal is to navigate from 's' to 'e' only using the building blocks (forward, backward, left, and right).

    Output Requirement:
    - Provide a JSON response with the list of instructions to solve the maze.

    Example Output:
    {{
        "instructions": ["forward", "left", "forward", "right", ...]
    }}

    If no path exists, return:
    {{
        "instructions": ["no path"]
    }}

    Only return the instructions list in the JSON response.
    """

    try:
        response = openai.ChatCompletion.create(
            # this is the fine tuned model output ID string
            model="ft:gpt-4o-mini-2024-07-18:personal:test5-5000entries:AcPUeYh0",
            messages=[{"role": "system", "content": "You are a helpful assistant."},
                      {"role": "user", "content": prompt}]
        )
        # Extract the response
        output = response['choices'][0]['message']['content']
        string_output = str(output)
        print("LLM Solution: " + string_output)
        instructions = json.loads(output)

        # Save the response as a JSON file
        with open(output_file, 'w') as json_file:
            json.dump(instructions, json_file, indent=4)
        print(f"Solution written to {output_file}")

        return string_output

    except Exception as e:
        print(f"Error solving the maze: {e}")

### A* Algorithm

In [59]:
@dataclass(order=True)
class PrioritizedItem:
    priority: float
    state: 'MazeState' = field(compare=False)

class MazeState:
    def __init__(self, x: int, y: int, direction: str, maze, path: List[str] = None):
        self.x = x
        self.y = y
        self.direction = direction
        self.maze = maze
        self.path = path or []
        self.size = maze.shape[0]

    def __hash__(self):
        return hash((self.x, self.y, self.direction))

    def __eq__(self, other):
        return (self.x, self.y, self.direction) == (other.x, other.y, other.direction)

    def get_possible_moves(self) -> List[Tuple[str, 'MazeState']]:
        moves = []
        directions = {
            'north': {'forward': (0, -1), 'left': 'west', 'right': 'east'},
            'south': {'forward': (0, 1), 'left': 'east', 'right': 'west'},
            'east': {'forward': (1, 0), 'left': 'north', 'right': 'south'},
            'west': {'forward': (-1, 0), 'left': 'south', 'right': 'north'}
        }

        current_dir_map = directions[self.direction]

        # Forward, Left, Right moves
        move_types = {
            "forward": (current_dir_map['forward'], self.direction),
            "left": (None, current_dir_map['left']),
            "right": (None, current_dir_map['right'])
        }

        for move_type, (delta, new_direction) in move_types.items():
            if delta is None:  # Rotation moves
                new_state = MazeState(
                    self.x, self.y, 
                    new_direction, 
                    self.maze, 
                    self.path + [move_type]
                )
                moves.append((move_type, new_state))
            else:
                new_x, new_y = self.x + delta[0], self.y + delta[1]
                
                # Check boundary and cell validity
                if (0 <= new_x < self.size and 0 <= new_y < self.size and self.maze.iloc[new_y][new_x] in ['o', 'e']):
                    new_state = MazeState(new_x, new_y, self.direction, self.maze, self.path + [move_type])
                    moves.append((move_type, new_state))

        return moves

def heuristic(state: MazeState, end: Tuple[int, int]) -> float:
    """Manhattan distance heuristic"""
    return abs(state.x - end[0]) + abs(state.y - end[1])

def solve_maze(maze):
    size = maze.shape[0]
    # print(f"Solving with A* algorithm")
    
    # Find start and end
    start = end = None
    for y in range(size):
        # print("column: " + str(y))
        for x in range(size):
            # print("cell: " + maze[y][x])
            if maze.iloc[y,x] == 's':
                start = (x, y)
            elif maze.iloc[y,x] == 'e':
                end = (x, y)
    
    # print(f"Start: {start}, End: {end}")

    # Default start direction (you can modify this if needed)
    start_state = MazeState(start[0], start[1], 'south', maze)
    
    # Priority queue for A* search
    open_set = []
    heapq.heappush(open_set, PrioritizedItem(0, start_state))
    
    # Tracking visited states to prevent cycles
    closed_set = set()

    while open_set:
        current_item = heapq.heappop(open_set)
        current_state = current_item.state
        # print(current_state.x, current_state.y, current_state.direction, current_state.path)

        # Check if reached end
        if current_state.x == end[0] and current_state.y == end[1]:
            return current_state.path

        # Avoid revisiting similar states
        state_key = hash(current_state)
        if state_key in closed_set:
            continue
        closed_set.add(state_key)

        # Explore possible moves
        for move, next_state in current_state.get_possible_moves():
            if hash(next_state) not in closed_set:
                # Calculate priority: path cost + heuristic
                priority = len(next_state.path) + heuristic(next_state, end)
                heapq.heappush(open_set, PrioritizedItem(priority, next_state))

    return ["no path"]  # No path found

# Read in CSV file and return the maze as a pandas DataFrame
def read_maze_from_csv(file):
    try:
        maze = pd.read_csv(file, header=None)

        # Check if the maze is square
        if maze.shape[0] != maze.shape[1]:
            print(f"Maze is not square. Shape is {maze.shape}, expected NxN.")
            sys.exit(1)

        # Check for unique start and end points
        start_count = (maze == 's').sum().sum()
        end_count = (maze == 'e').sum().sum()
        
        if start_count != 1:
            print(f"Invalid number of start points. Found {start_count}, expected 1.")
            sys.exit(1)
        
        if end_count != 1:
            print(f"Invalid number of end points. Found {end_count}, expected 1.")
            sys.exit(1)

        return maze
    
    except Exception as e:
        print(f"Error reading the maze file: {e}")
        return None

def run_A_STAR(maze_file, output_json: str):
    maze = read_maze_from_csv(maze_file)
    print("Maze loaded:")
    print(maze.to_csv(index=False, header=False))
    
    solution = solve_maze(maze)
    formatted_solution = {"instructions": solution}
    print()
    print("A*  Solution: " + str(formatted_solution))

    if formatted_solution:
        with open(output_json, 'w') as f:
            json.dump(formatted_solution, f)
        print(f"Solution written to {output_json}")
        print()
    else:
        print("No solution found")
    
    return str(formatted_solution)
        
